# AutoHighlight: AI-Powered Video Highlights Generation

Welcome! **AutoHighlight** is an innovative AI-powered pipeline that automatically generates compelling video highlights from any video content. By combining Azure Content Understanding with OpenAI's advanced reasoning capabilities, it intelligently identifies the most engaging moments and creates professional highlight reels tailored to your specific needs.

---

## What AutoHighlight Does

Transform hours of video content into captivating highlights within minutes:

- **Smart Content Analysis:** Utilizes Azure Content Understanding to analyze every frame and identify key moments.
- **AI-Powered Selection:** Leverages OpenAI o3's reasoning to select the most compelling clips based on your preferences.
- **Automated Editing:** Creates seamless highlight videos with professional transitions and timing.
- **Personalized Results:** Tailors highlights to specific content types (sports, presentations, events) and your custom preferences.
- **Production Ready:** Robust pipeline suitable for content creators, marketers, educators, and media professionals.

Perfect for creating highlight reels from sports events, keynote presentations, product demos, educational content, entertainment shows, and more!

---

## Key Features

- **Multi-Domain Support:** Pre-configured schemas for sports (soccer, basketball, football), presentations, events, and custom content types.
- **Intelligent Timestamp Selection:** AI reasoning identifies the most impactful moments based on content analysis.
- **Flexible Output Control:** Customize highlight duration, clip density, transitions, and personalization preferences.
- **Professional Video Processing:** Video editing supporting various resolutions and effects.
- **User-Friendly Interface:** Jupyter notebook with step-by-step guidance and clear documentation.

The pipeline uses the latest Azure AI Content Understanding API (`2025-05-01-preview`) combined with OpenAI's most advanced reasoning models.

## Pre-requisites
1. Please follow the [README](../README.md#configure-azure-ai-service-resource) to create the essential resources required for this sample.
2. Install the required packages.

In [ ]:
%pip install -r ../requirements.txt

> Note: 
> - We've tested with the GPT `o3` model. However, API calls might be blocked by the content filter. Please consider modifying the content filter settings of your GPT model deployment if needed. [How to configure content filters](https://learn.microsoft.com/en-us/azure/ai-foundry/openai/how-to/content-filters?utm_source=chatgpt.com)
> - All customers have the ability to modify the content filters and configure the severity thresholds (low, medium, high). Approval is required for turning the content filters partially or fully off. Managed customers only may apply for full content filtering control via this form: [Azure OpenAI Limited Access Review: Modified Content Filters](https://ncv.microsoft.com/uEfCgnITdR). At this time, it is not possible to become a managed customer.
>   - This form is also used to register for approval to use Azure AI Content Understanding with content filtering turned off. Subscription IDs approved for Modified Content Filtering will impact Azure AI Content Understanding output. To learn more about Azure AI Content Understanding, please see the documentation [here](https://learn.microsoft.com/en-us/azure/ai-services/content-understanding/overview).

## Azure AAD Login
### Option 1: Login with Azure Developer CLI (azd)
If you haven't already, please authenticate by running **`azd auth login`** through the terminal. This credential is used to validate your access to the resources you defined in .env file and this notebook.

Log in to Azure:

```bash
azd auth login
```

If this command doesn’t work, try the device code login:

```bash
azd auth login --use-device-code
```


### Option 2: Login with Azure CLI (az)
If you encounter authentication issue when using Azure Developer CLI (azd), please try login with Azure CLI (az).

Please ensure you have the Azure CLI installed on your system. To install, run this in the terminal:
```bash
curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash
```

Log in to Azure:
```bash
az login
```

### Option 3: Use Endpoint with API key
If you would like to use endpoint with API key, please add **AZURE_AI_SERVICE_API_KEY** and **AZURE_OPENAI_API_API_KEY** into [.env](.env).

> ⚠️ Note: Using a subscription key works, but using a token provider with Azure Active Directory (AAD) is safer and strongly recommended for production environments.

## Load Environment Variables and Create Clients

> The [AzureContentUnderstandingClient](../python/content_understanding_client.py) is a utility class providing functions to interact with the Content Understanding API. Until the official release of the Content Understanding SDK, it functions as a lightweight SDK. Please fill in the constants **AZURE_AI_SERVICE_ENDPOINT** and **AZURE_AI_SERVICE_API_VERSION** with your Azure AI Service information. Optionally, provide **AZURE_AI_SERVICE_API_KEY** if your setup requires key-based authentication.

> ⚠️ Important:
Please update the code below to match your Azure authentication method. Look for the `# IMPORTANT` comments and modify those sections accordingly. Skipping this step may cause the sample to not run correctly.

> ⚠️ Note: Using a subscription key works, but using a token provider with Azure Active Directory (AAD) is safer and highly recommended for production environments.

In [ ]:
from dotenv import load_dotenv
import os
import sys
from pathlib import Path

from azure.identity import DefaultAzureCredential, get_bearer_token_provider

parent_dir = Path(Path.cwd()).parent
# Add the parent directory to the path to use shared modules
sys.path.append(str(parent_dir))
from python.utility import OpenAIAssistant
from python.content_understanding_client import AzureContentUnderstandingClient

load_dotenv(dotenv_path=".env", override=True)

AZURE_AI_SERVICE_ENDPOINT = os.getenv("AZURE_AI_SERVICE_ENDPOINT")
AZURE_AI_SERVICE_API_VERSION = os.getenv("AZURE_AI_SERVICE_API_VERSION", "2025-05-01-preview")
# IMPORTANT: Replace with your actual subscription key or set it in the ".env" file if not using token authentication.
AZURE_AI_SERVICE_API_KEY = os.getenv("AZURE_AI_API_KEY")

AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION", "2024-12-01-preview")
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
# IMPORTANT: Replace with your actual subscription key or set it in the ".env" file if not using token authentication.
AZURE_OPENAI_API_API_KEY = os.getenv("AZURE_AI_API_KEY", None)

# Create an OpenAI Assistant to interact with Azure OpenAI
openai_assistant = OpenAIAssistant(
    aoai_end_point=AZURE_OPENAI_ENDPOINT,
    aoai_api_version=AZURE_OPENAI_API_VERSION,
    deployment_name=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
    aoai_api_key=AZURE_OPENAI_API_API_KEY,
)

# Create a token provider using DefaultAzureCredential for AAD authentication for AzureContentUnderstandingClient
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

# Create the Content Understanding (CU) client
cu_client = AzureContentUnderstandingClient(
    endpoint=AZURE_AI_SERVICE_ENDPOINT,
    api_version=AZURE_AI_SERVICE_API_VERSION,
    # IMPORTANT: Comment out token_provider if using subscription key
    token_provider=token_provider,
    # IMPORTANT: Uncomment this if using subscription key
    # subscription_key=AZURE_AI_SERVICE_API_KEY,
    x_ms_useragent="azure-ai-content-understanding-python/video_chapters_dynamic",  # This header is used for sample usage telemetry; please comment out this line if you wish to opt out.
)

## VIDEO CONFIGURATION

Use the following variables to configure the auto-highlight generation tailored to your video content and preferences.

Key settings you control:

- **🎥 Video source:** Path to your video file
- **🎬 Content type:** Type of video you're analyzing (sports, keynote, etc.)
- **⏱️ Target length:** Desired length of the final highlight video
- **🎯 Style preferences:** Clip density, transitions, effects
- **🚀 Personalization:** Any specific expectations you want your highlights to reflect

> Important: Please specify the `SOURCE_VIDEO_PATH` and select the correct `VIDEO_TYPE` before running the notebook. The `VIDEO_TYPE` determines what events the AI looks for (e.g., "soccer" detects goals and exciting plays; "keynote" identifies key quotes and audience reactions).

In [ ]:
# 🎥 VIDEO CONFIGURATION

import time
import uuid
# Replace with your actual video file path
SOURCE_VIDEO_PATH = "../data/FlightSimulator.mp4"  # Please update this path to your video file

# 📁 OUTPUT DIRECTORY (will create output folder if it doesn't exist)
OUTPUT_DIR = parent_dir / "output"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# 🎬 SCHEMA CONFIGURATION (schema types available in ./schemas/ folder)
VIDEO_TYPE = "demo"  # Options: "soccer", "basketball", "football", etc.

# Path to the analyzer template directory (auto-configured based on VIDEO_TYPE)
ANALYZER_DIR = "../analyzer_templates/auto_highlight_analyzers"

# 🎯 HIGHLIGHT GENERATION PARAMETERS
TARGET_DURATION_S = 20             # Target duration for the final highlight video in seconds
CLIP_DENSITY = 1.0                 # Clip density to include (1.0 = normal, >1.0 = more clips, <1.0 = fewer clips)
PERSONALIZATION = "none"           # Any specific expectations for the highlight
TRANSITION_TYPE = "cut"            # Options: "cut", "fade"
SPEED_RAMP = False                 # Apply speed ramping if True
ADD_CAPTIONS = False               # Add captions (requires additional setup, default is False)
RESOLUTION = 720                   # Output resolution height (e.g., 720p, 1080p)
HUMAN_IN_THE_LOOP_REVIEW = False   # Pause for human review of analyzer generation if True, This is using Tkinter GUI which may not work on all environments 

# Analyzer ID configuration
ANALYZER_ID = f"highlight-analyzer-{str(uuid.uuid4())[:8]}-{int(time.time())}"  # Unique analyzer ID using UUID


if not os.path.exists(SOURCE_VIDEO_PATH):
    raise ValueError(f"❌ Video file not found: {SOURCE_VIDEO_PATH}")

print("✅ Configuration loaded successfully!")
print("🔧 Configuration Details:")
print(f"Source Video: {SOURCE_VIDEO_PATH}")
print(f"Output Directory: {OUTPUT_DIR}")
print(f"Analyzer Directory: {ANALYZER_DIR}")
print(f"Video Type: {VIDEO_TYPE}")
print(f"Target Duration: {TARGET_DURATION_S} seconds")
print(f"Analyzer ID: {ANALYZER_ID}")

## 1. Generate and Activate Schema
*What this does:* Creates a custom Azure AI analyzer that uses OpenAI reasoning to understand your specific video content and identify highlight-worthy moments based on your preferences.

The schema generation process uses your VIDEO_TYPE, CLIP_DENSITY, TARGET_DURATION_S, and PERSONALIZATION settings to build an intelligent analyzer that knows exactly what to look for in your video (e.g., goals in soccer, key quotes in presentations, exciting moments in gameplay).

### Schema Review GUI Instructions
⚠️ Important:
This application uses Tkinter GUI dialogs for human-in-the-loop review.
Tkinter requires a graphical display and may not work in headless environments such as GitHub Codespaces, remote servers, Docker containers, or WSL without GUI support. If you encounter `TclError: no display name and no $DISPLAY environment variable`, you can disable human-in-the-loop review by setting `HUMAN_IN_THE_LOOP_REVIEW` to `False`.

If you set `HUMAN_IN_THE_LOOP_REVIEW` to `True`, the Schema Review GUI will launch. Here is how to use the Schema Review GUI:

1. **Review Schema Fields:**
   - Each row displays the field name, its type, and its description.
   - Checkboxes next to each field allow you to include or exclude it from the schema.
     - Uncheck the box next to any field you want to exclude from the final schema.
     - Checked fields will be included; unchecked ones will be removed.
2. **Save Button:**
   - Click "Save" to:
     - Save your current field selections to the schema file.
     - Refresh the list of fields to reflect your changes.
     - Receive a confirmation popup indicating the schema was saved.
3. **Add Manual Field:**
   - Click "Add Manual Field" to open a dialog allowing you to add a new field.
     - Enter the field name.
     - Select the field type (string, date, time, number, integer).
     - Choose the method (classify or generate).
     - Enter a description.
     - If "classify" is selected, you can provide enum options (comma-separated).
   - Click "OK" to add the field to the schema and update the list.
4. **Add GPT Field:**
   - Click "Add GPT Field" to let GPT suggest a field based on your description.
     - Provide a description of the field you want.
     - The assistant will generate and add the field to the schema.
     - (Requires an OpenAI assistant to be provided.)
5. **Done Button:**
   - Click "Done" when you have finished reviewing.
     - The schema will be saved with your final selection.
     - The GUI will close.
     - A confirmation popup will appear.

**Tips:**
- Feel free to add, remove, or edit fields as many times as needed before clicking "Done".
- Use "Save" to checkpoint your progress without closing the window.
- If you make a mistake, you can re-add fields or adjust your selections before finishing.

In [ ]:
from python.auto_highlight_utility import activate_schema

schema_path = activate_schema(
    video_type=VIDEO_TYPE,
    analyzer_dir=ANALYZER_DIR,
    openai_assistant=openai_assistant,
    output_dir=OUTPUT_DIR,
    clip_density=CLIP_DENSITY,
    target_duration_s=TARGET_DURATION_S, 
    personalization=PERSONALIZATION,
    human_in_the_loop_review=HUMAN_IN_THE_LOOP_REVIEW)

print(f"Schema activated: {schema_path}")

# Display schema content
with open(schema_path, 'r') as f:
    import json
    print(json.dumps(json.load(f), indent=2))

## 2. Analyze Video
Now, submit the video to Azure Content Understanding (CU) for analysis using the custom schema you just generated. This step may take some time depending on the length of the video.

### Create a Custom Analyzer

In [ ]:
import json

# Use the client to create an analyzer
response = cu_client.begin_create_analyzer(ANALYZER_ID, analyzer_template_path=schema_path)
result = cu_client.poll_result(response)

print(json.dumps(result, indent=2))

### Use the created analyzer to extract video content

In [ ]:
# Submit the video for content analysis
response = cu_client.begin_analyze(ANALYZER_ID, file_location=SOURCE_VIDEO_PATH)

# Wait for the analysis to complete and retrieve the content analysis result
video_cu_result = cu_client.poll_result(
    response, timeout_seconds=3600)  # 1 hour timeout for long videos

# Print the content analysis result
print(f"Video Content Understanding result:", video_cu_result)

cu_result_filename = f"{os.path.basename(SOURCE_VIDEO_PATH)}.json"
cu_result_path = os.path.join(OUTPUT_DIR, cu_result_filename)

# Save results
with open(cu_result_path, 'w', encoding='utf-8') as f:
    json.dump(video_cu_result, f, indent=2)

print(f"Video analysis complete. Results saved to: {cu_result_path}")

### Optional - Delete the analyzer if it is no longer needed
Note: In production environments, analyzers are typically kept for reuse rather than deleted.

In [ ]:
cu_client.delete_analyzer(ANALYZER_ID)

## 3. Parse and Pre-filter Segments

Using the output from Azure CU, which describes all the video frames and their content, we apply rule-based filtering to remove less significant frames. This filtering is based on the schema's scoring criteria and creates a simplified list of potential highlight clips.

In [ ]:
from python.auto_highlight_utility import get_filtered_segments

prefiltered_segments_path = get_filtered_segments(input_path=cu_result_path)
print(f"Segments parsed and pre-filtered. Results saved to: {prefiltered_segments_path}")

# Display the pre-filtered segments
with open(prefiltered_segments_path, 'r') as f:
    segments_data = json.load(f)
    print(json.dumps(segments_data, indent=2))

## 4. Highlight Edits Filtering
Next, we send the parsed segments to OpenAI o3 for advanced reasoning and intelligent timestamp selection. The AI analyzes all potential highlight clips and selects specific timestamps that will create the most compelling custom highlights tailored to your preferences, while meeting the `TARGET_DURATION_S` and arranging them in an optimal narrative order.

In [ ]:
from python.utility import get_highlight_plan

result = get_highlight_plan(
    openai_assistant=openai_assistant,
    segments=segments_data,
    video_type=VIDEO_TYPE,
    target_duration_s=TARGET_DURATION_S,
    clip_density=CLIP_DENSITY,
    personalization=PERSONALIZATION,
)

In [ ]:
result_dict = result.model_dump()

# Save to JSON file
highlight_plan_file = "highlight_plan.json"
highlight_plan_path = os.path.join(OUTPUT_DIR, highlight_plan_file)
with open(highlight_plan_path, "w", encoding="utf-8") as f:
    json.dump(result_dict, f, indent=2, ensure_ascii=False)

print(f"Saved JSON to {highlight_plan_path}")
print(result_dict)

## 5. Stitch Video Clips
Based on the highlight plan generated by OpenAI's reasoning, we now stitch together the custom user highlights. This final step takes the intelligently selected timestamps and creates a seamless highlight video tailored to your preferences using FFmpeg for video editing.

In [ ]:
# Video Stitching using FFmpeg
from python.video_modification_utility import stitch_video

# Define output path for the highlight video
OUTPUT_HIGHLIGHT_PATH = os.path.join(OUTPUT_DIR, "highlight.mp4")

print("🎬 Running video stitching...")
print(f"📄 Input video: {SOURCE_VIDEO_PATH}")
print(f"📄 Highlight plan: {highlight_plan_path}")
print(f"📄 Output video: {OUTPUT_HIGHLIGHT_PATH}")

try:
    result = stitch_video(
        video_path=SOURCE_VIDEO_PATH,
        plan_path=highlight_plan_path,
        output_path=OUTPUT_HIGHLIGHT_PATH,
        transition=TRANSITION_TYPE,
        speed_ramp=SPEED_RAMP,
        resolution=RESOLUTION
    )
    
    if result:
        print(f"✅ SUCCESS: Highlight video created at {result}")
        
        # Check file info
        if os.path.exists(result):
            size_mb = os.path.getsize(result) / (1024*1024)
            print(f"   📁 File size: {size_mb:.2f} MB")
            print(f"   📁 Full path: {result}")
        print(f"\n🎉 Highlight generation pipeline completed successfully!")
    else:
        print("❌ Video stitching failed")
        
except Exception as e:
    print(f"❌ ERROR during video stitching: {e}")
    import traceback
    traceback.print_exc()